In [6]:
# Auto reload modules
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import sys
from typing import List
import numpy as np
import pandas as pd
import copy
from tqdm.notebook import tqdm

sys.path.append("/home/pillowbeast/Documents/weac")

from weac.analysis import CriteriaEvaluator, CoupledCriterionResult, SSERRResult
from weac.core.system_model import SystemModel
from weac.components import ModelInput, Segment, ScenarioConfig, WeakLayer, CriteriaConfig, Layer
from weac.utils.snowpilot_parser import SnowPilotParser

In [8]:
data_dir = "data/raw/snowpilot"

In [9]:
number_of_files = 10000

# Process multiple files
file_paths = []
for directory in os.listdir(data_dir):
    for file in os.listdir(f"{data_dir}/{directory}"):
        if file.endswith(".xml"):
            file_paths.append(f"{data_dir}/{directory}/{file}")

paths: List[str] = []
parsers: List[SnowPilotParser] = []

for file_path in file_paths[:number_of_files]:
    snowpilot_parser = SnowPilotParser(file_path)
    if not snowpilot_parser.snowpit.core_info.location.pit_near_avalanche:
        paths.append(file_path)
        parsers.append(snowpilot_parser)

print(f"\nFound {len(paths)} files without avalanche")


Found 9665 files without avalanche


In [10]:
import numpy as np

spacing = 10
depth_profile = np.arange(0, 4000, spacing)
densities = np.zeros((len(parsers), len(depth_profile)))

error_count = 0
for i, parser in enumerate(parsers):
    try:
        layers, _ = parser.extract_layers()
    except Exception as e:
        error_count += 1
        densities[i, :] = np.nan
        print(f"Error extracting layers for {parser.snowpit.core_info.location.pit_near_avalanche_location}: {e}")
        continue
    heights = np.cumsum([layer.h for layer in layers])
    heights = np.concatenate([[0], heights])
    heights_idx = np.round(heights/spacing).astype(int)
    for j, height in enumerate(heights_idx):
        if j > 0:
            densities[i, int(heights_idx[j-1]):int(heights_idx[j])] = layers[j-1].rho

print(f"Error count: {error_count}")

Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Excluding SnowPit from calculations.
Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Excluding SnowPit from calculations.
Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Excluding SnowPit from calculations.
Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Excluding SnowPit from calculations.
Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Excluding SnowPit from calculations.
Error extracting layers for None: Layer is missing density information; density profile, hand hardness and grain type are all missing. Exclu

In [11]:
import numpy as np

# Calculate the average density at each depth
densities = np.where(densities == 0, np.nan, densities)
# Calculate the average density at each depth
average_density = np.nanmean(densities, axis=0)
print(average_density)



[137.06439094 136.66971308 136.57830987 137.44609859 138.15601684
 140.72867205 141.43036535 142.4325297  144.46279799 145.66975004
 150.76430275 151.9769512  154.70593219 156.67307646 158.40422919
 163.90928949 165.2565934  167.15659484 169.27333552 170.63456235
 176.25023037 177.29561765 178.93349723 180.85882402 182.01256574
 185.91124411 187.10287662 188.49625177 189.96924118 191.38841371
 196.52972956 197.32057174 197.94207446 199.65164811 200.97547909
 204.36606591 205.11618762 205.67487414 206.94071207 207.79497812
 210.75889087 211.16812425 211.97711477 212.87148478 213.8426599
 215.79357754 216.38364188 216.9549473  217.7599194  218.25274945
 220.6783367  220.96027852 221.76017622 222.11597563 223.00619129
 224.97543904 225.59273743 226.05453106 227.14395661 227.55742361
 229.10922746 229.46513011 230.17193687 230.77376283 230.88327701
 232.34128049 232.52322612 232.9328204  232.95709883 233.26870489
 234.18202629 234.15657456 234.61884737 234.70984051 234.74875072
 236.427305

In [12]:
# Setup standard values
phi = 0.0
scenario_config = ScenarioConfig(system_type="skier", phi=phi)
weaklayer = WeakLayer(rho=125, h=20, E=1.0, sigma_c=6.16, tau_c=5.09)
segments = [
    Segment(length=10000, has_foundation=True, m=0.0),
    Segment(
        length=10000,
        has_foundation=True,
        m=0.0,
    ),
]
criteria_config = CriteriaConfig()
criteria_evaluator = CriteriaEvaluator(criteria_config)

In [13]:
def eval_weac_over_layers(layers: List[Layer], scenario_config: ScenarioConfig, segments: list[Segment], weaklayer: WeakLayer, spacing=100):
    data_rows = []
    heights = np.cumsum([layer.h for layer in layers])
    # space evenly and append the last height
    wl_depths = np.arange(spacing, heights[-1], spacing).tolist()
    wl_depths.append(heights[-1])
    
    layers_copy = copy.deepcopy(layers)
    for i, wl_depth in tqdm(enumerate(wl_depths), total=len(wl_depths), desc="Processing weak layers", leave=False):
        # only keep layers above the spacing
        mask = heights <= wl_depth
        new_layers = [layer for layer, keep in zip(layers_copy, mask) if keep]
        # Add truncated layer if needed
        depth = np.sum([layer.h for layer in new_layers]) if new_layers else 0.0
        if depth < wl_depth:
            additional_layer = copy.deepcopy(layers_copy[len(new_layers) if new_layers else 0])
            additional_layer.h = wl_depth - depth
            new_layers.append(additional_layer)
        
        model_input = ModelInput(
            weak_layer=weaklayer,
            layers=new_layers,
            scenario_config=scenario_config,
            segments=segments,
        )
        system = SystemModel(model_input=model_input)
        
        cc_result: CoupledCriterionResult = criteria_evaluator.evaluate_coupled_criterion(system, print_call_stats=False)
        sserr_result: SSERRResult = criteria_evaluator.evaluate_SSERR(system, vertical=False, print_call_stats=False)

        data_rows.append({
            "wl_depth": wl_depth,
            "impact_criterion": cc_result.initial_critical_skier_weight,
            "coupled_criterion": cc_result.critical_skier_weight,
            "sserr_result": sserr_result.SSERR,
            "touchdown_distance": sserr_result.touchdown_distance,
        })
    return data_rows

In [14]:
layers = []
for i, depth in enumerate(depth_profile):
    layers.append(Layer(rho=average_density[i], h=spacing))

print(len(layers))

# data_rows = eval_weac_over_layers(layers, scenario_config, segments, weaklayer, spacing=spacing)
# dataframe = pd.DataFrame(data_rows)
# dataframe.to_csv("data/misc/average_density_profile.csv", index=False)

400


In [15]:
import pandas as pd
import plotly.graph_objects as go

dataframe = pd.read_csv("data/misc/average_density_profile.csv")
depths = dataframe["wl_depth"]
impact_criterion = dataframe["impact_criterion"]
coupled_criterion = dataframe["coupled_criterion"]
sserr_result = dataframe["sserr_result"]
touchdown_distance = dataframe["touchdown_distance"]

densities = [layer.rho for layer in layers]

fig = go.Figure()
fig.add_trace(go.Scatter(x=depths, y=impact_criterion, mode='lines', name='Impact Criterion'))
fig.add_trace(go.Scatter(x=depths, y=coupled_criterion, mode='lines', name='Coupled Criterion'))
fig.add_trace(go.Scatter(x=depths, y=sserr_result, mode='lines', name='SSERR'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=densities, y=depths, mode='lines', name='Density'))
fig.show()